In [1]:
import re
import numpy as np
import collections
from sklearn import metrics
from sklearn.cross_validation import train_test_split
import tensorflow as tf
import pandas as pd
from unidecode import unidecode
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import time

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
rules_normalizer = {
    'experience': 'pengalaman',
    'bagasi': 'bagasi',
    'kg': 'kampung',
    'kilo': 'kilogram',
    'g': 'gram',
    'grm': 'gram',
    'k': 'okay',
    'abgkat': 'abang dekat',
    'abis': 'habis',
    'ade': 'ada',
    'adoi': 'aduh',
    'adoii': 'aduhh',
    'aerodarat': 'kapal darat',
    'agkt': 'angkat',
    'ahh': 'ah',
    'ailior': 'air liur',
    'airasia': 'air asia x',
    'airasiax': 'penerbangan',
    'airline': 'penerbangan',
    'airlines': 'penerbangan',
    'airport': 'lapangan terbang',
    'airpot': 'lapangan terbang',
    'aje': 'sahaja',
    'ajelah': 'sahajalah',
    'ajer': 'sahaja',
    'ak': 'aku',
    'aq': 'aku',
    'all': 'semua',
    'ambik': 'ambil',
    'amek': 'ambil',
    'amer': 'amir',
    'amik': 'ambil',
    'ana': 'saya',
    'angkt': 'angkat',
    'anual': 'tahunan',
    'apapun': 'apa pun',
    'ape': 'apa',
    'arab': 'arab',
    'area': 'kawasan',
    'aritu': 'hari itu',
    'ask': 'tanya',
    'astro': 'astro',
    'at': 'pada',
    'attitude': 'sikap',
    'babi': 'khinzir',
    'back': 'belakang',
    'bag': 'beg',
    'bang': 'abang',
    'bangla': 'bangladesh',
    'banyk': 'banyak',
    'bard': 'pujangga',
    'bargasi': 'bagasi',
    'bawak': 'bawa',
    'bawanges': 'bawang',
    'be': 'jadi',
    'behave': 'berkelakuan baik',
    'belagak': 'berlagak',
    'berdisiplin': 'berdisplin',
    'berenti': 'berhenti',
    'beskal': 'basikal',
    'bff': 'rakan karib',
    'bg': 'bagi',
    'bgi': 'bagi',
    'biase': 'biasa',
    'big': 'besar',
    'bike': 'basikal',
    'bile': 'bila',
    'binawe': 'binatang',
    'bini': 'isteri',
    'bkn': 'bukan',
    'bla': 'bila',
    'blom': 'belum',
    'bnyak': 'banyak',
    'body': 'tubuh',
    'bole': 'boleh',
    'boss': 'bos',
    'bowling': 'boling',
    'bpe': 'berapa',
    'brand': 'jenama',
    'brg': 'barang',
    'briefing': 'taklimat',
    'brng': 'barang',
    'bro': 'abang',
    'bru': 'baru',
    'bruntung': 'beruntung',
    'bsikal': 'basikal',
    'btnggjwb': 'bertanggungjawab',
    'btul': 'betul',
    'buatlh': 'buatlah',
    'buh': 'letak',
    'buka': 'buka',
    'but': 'tetapi',
    'bwk': 'bawa',
    'by': 'dengan',
    'byr': 'bayar',
    'bz': 'sibuk',
    'camera': 'kamera',
    'camni': 'macam ini',
    'cane': 'macam mana',
    'cant': 'tak boleh',
    'carakerja': 'cara kerja',
    'care': 'jaga',
    'cargo': 'kargo',
    'cctv': 'kamera litar tertutup',
    'celako': 'celaka',
    'cer': 'cerita',
    'cheap': 'murah',
    'check': 'semak',
    'ciput': 'sedikit',
    'cite': 'cerita',
    'citer': 'cerita',
    'ckit': 'sikit',
    'ckp': 'cakap',
    'class': 'kelas',
    'cm': 'macam',
    'cmni': 'macam ini',
    'cmpak': 'campak',
    'committed': 'komited',
    'company': 'syarikat',
    'complain': 'aduan',
    'corn': 'jagung',
    'couldnt': 'tak boleh',
    'cr': 'cari',
    'crew': 'krew',
    'cube': 'cuba',
    'cuma': 'cuma',
    'curinyaa': 'curinya',
    'cust': 'pelanggan',
    'customer': 'pelanggan',
    'd': 'di',
    'da': 'dah',
    'dn': 'dan',
    'dahh': 'dah',
    'damaged': 'rosak',
    'dapek': 'dapat',
    'day': 'hari',
    'dazrin': 'dazrin',
    'dbalingnya': 'dibalingnya',
    'de': 'ada',
    'deep': 'dalam',
    'deliberately': 'sengaja',
    'depa': 'mereka',
    'dessa': 'desa',
    'dgn': 'dengan',
    'dh': 'dah',
    'didunia': 'di dunia',
    'diorang': 'mereka',
    'diorng': 'mereka',
    'direct': 'secara terus',
    'diving': 'junam',
    'dkt': 'dekat',
    'dlempar': 'dilempar',
    'dlm': 'dalam',
    'dlt': 'padam',
    'dlu': 'dulu',
    'done': 'siap',
    'dont': 'jangan',
    'dorg': 'mereka',
    'dpermudhkn': 'dipermudahkan',
    'dpt': 'dapat',
    'dr': 'dari',
    'dri': 'dari',
    'dsb': 'dan sebagainya',
    'dy': 'dia',
    'educate': 'mendidik',
    'ensure': 'memastikan',
    'everything': 'semua',
    'ewahh': 'wah',
    'expect': 'sangka',
    'fb': 'facebook',
    'fired': 'pecat',
    'first': 'pertama',
    'fkr': 'fikir',
    'flight': 'kapal terbang',
    'for': 'untuk',
    'free': 'percuma',
    'friend': 'kawan',
    'fyi': 'untuk pengetahuan anda',
    'gantila': 'gantilah',
    'gantirugi': 'ganti rugi',
    'gentlemen': 'lelaki budiman',
    'gerenti': 'jaminan',
    'gile': 'gila',
    'gk': 'juga',
    'gnti': 'ganti',
    'go': 'pergi',
    'gomen': 'kerajaan',
    'goment': 'kerajaan',
    'good': 'baik',
    'ground': 'tanah',
    'guarno': 'macam mana',
    'hampa': 'mereka',
    'hampeh': 'teruk',
    'hanat': 'jahanam',
    'handle': 'kawal',
    'handling': 'kawalan',
    'hanta': 'hantar',
    'haritu': 'hari itu',
    'hate': 'benci',
    'have': 'ada',
    'hawau': 'celaka',
    'henpon': 'telefon',
    'heran': 'hairan',
    'him': 'dia',
    'his': 'dia',
    'hmpa': 'mereka',
    'hntr': 'hantar',
    'hotak': 'otak',
    'hr': 'hari',
    'i': 'saya',
    'hrga': 'harga',
    'hrp': 'harap',
    'hu': 'sedih',
    'humble': 'merendah diri',
    'ibon': 'ikon',
    'ichi': 'inci',
    'idung': 'hidung',
    'if': 'jika',
    'ig': 'instagram',
    'iklas': 'ikhlas',
    'improve': 'menambah baik',
    'in': 'masuk',
    'isn t': 'tidak',
    'isyaallah': 'insyallah',
    'ja': 'sahaja',
    'japan': 'jepun',
    'jd': 'jadi',
    'je': 'saja',
    'jee': 'saja',
    'jek': 'saja',
    'jepun': 'jepun',
    'jer': 'saja',
    'jerr': 'saja',
    'jez': 'saja',
    'jg': 'juga',
    'jgk': 'juga',
    'jgn': 'jangan',
    'jgnla': 'janganlah',
    'jibake': 'celaka',
    'jjur': 'jujur',
    'job': 'kerja',
    'jobscope': 'skop kerja',
    'jogja': 'jogjakarta',
    'jpam': 'jpam',
    'jth': 'jatuh',
    'jugak': 'juga',
    'ka': 'ke',
    'kalo': 'kalau',
    'kalu': 'kalau',
    'kang': 'nanti',
    'kantoi': 'temberang',
    'kasi': 'beri',
    'kat': 'dekat',
    'kbye': 'ok bye',
    'kearah': 'ke arah',
    'kecik': 'kecil',
    'keja': 'kerja',
    'keje': 'kerja',
    'kejo': 'kerja',
    'keksongan': 'kekosongan',
    'kemana': 'ke mana',
    'kene': 'kena',
    'kenekan': 'kenakan',
    'kesah': 'kisah',
    'ketempat': 'ke tempat',
    'kije': 'kerja',
    'kijo': 'kerja',
    'kiss': 'cium',
    'kite': 'kita',
    'kito': 'kita',
    'kje': 'kerja',
    'kjr': 'kerja',
    'kk': 'okay',
    'kmi': 'kami',
    'kt': 'kat',
    'tlg': 'tolong',
    'kl': 'kuala lumpur',
    'klai': 'kalau',
    'klau': 'kalau',
    'klia': 'klia',
    'klo': 'kalau',
    'klu': 'kalau',
    'kn': 'kan',
    'knapa': 'kenapa',
    'kne': 'kena',
    'ko': 'kau',
    'kompom': 'sah',
    'korang': 'kamu semua',
    'korea': 'korea',
    'korg': 'kamu semua',
    'kot': 'mungkin',
    'krja': 'kerja',
    'ksalahan': 'kesalahan',
    'kta': 'kita',
    'kuar': 'keluar',
    'kut': 'mungkin',
    'la': 'lah',
    'laa': 'lah',
    'lahabau': 'celaka',
    'lahanat': 'celaka',
    'lainda': 'lain dah',
    'lak': 'pula',
    'last': 'akhir',
    'le': 'lah',
    'leader': 'ketua',
    'leave': 'pergi',
    'ler': 'lah',
    'less': 'kurang',
    'letter': 'surat',
    'lg': 'lagi',
    'lgi': 'lagi',
    'lngsong': 'langsung',
    'lol': 'hehe',
    'lorr': 'lah',
    'low': 'rendah',
    'lps': 'lepas',
    'luggage': 'bagasi',
    'lumbe': 'lumba',
    'lyak': 'layak',
    'maap': 'maaf',
    'maapkan': 'maafkan',
    'mahai': 'mahal',
    'mampos': 'mampus',
    'mart': 'kedai',
    'mau': 'mahu',
    'mcm': 'macam',
    'mcmtu': 'macam itu',
    'memerlukn': 'memerlukan',
    'mengembirakan': 'menggembirakan',
    'mengmbilnyer': 'mengambilnya',
    'mengtasi': 'mengatasi',
    'mg': 'memang',
    'mihak': 'memihak',
    'min': 'admin',
    'mingu': 'minggu',
    'mintak': 'minta',
    'mjtuhkn': 'menjatuhkan',
    'mkyong': 'mak yong',
    'mlibatkn': 'melibatkan',
    'mmg': 'memang',
    'mmnjang': 'memanjang',
    'mmpos': 'mampus',
    'mn': 'mana',
    'mna': 'mana',
    'mntak': 'minta',
    'mntk': 'minta',
    'mnyusun': 'menyusun',
    'mood': 'suasana',
    'most': 'paling',
    'mr': 'tuan',
    'msa': 'masa',
    'msia': 'malaysia',
    'mst': 'mesti',
    'mu': 'awak',
    'much': 'banyak',
    'muko': 'muka',
    'mum': 'emak',
    'n': 'dan',
    'nah': 'nah',
    'nanny': 'nenek',
    'napo': 'kenapa',
    'nati': 'nanti',
    'ngan': 'dengan',
    'ngn': 'dengan',
    'ni': 'ini',
    'nie': 'ini',
    'nii': 'ini',
    'nk': 'nak',
    'nmpk': 'nampak',
    'nye': 'nya',
    'ofis': 'pejabat',
    'ohh': 'oh',
    'oii': 'hoi',
    'one': 'satu',
    'online': 'dalam talian',
    'or': 'atau',
    'org': 'orang',
    'orng': 'orang',
    'otek': 'otak',
    'p': 'pergi',
    'paid': 'dah bayar',
    'palabana': 'kepala otak',
    'pasni': 'lepas ini',
    'passengers': 'penumpang',
    'passengger': 'penumpang',
    'pastu': 'lepas itu',
    'pd': 'pada',
    'pegi': 'pergi',
    'pekerje': 'pekerja',
    'pekrja': 'pekerja',
    'perabih': 'perabis',
    'perkerja': 'pekerja',
    'pg': 'pergi',
    'phuii': 'puih',
    'pikir': 'fikir',
    'pilot': 'juruterbang',
    'pk': 'fikir',
    'pkerja': 'pekerja',
    'pkerjaan': 'pekerjaan',
    'pki': 'pakai',
    'please': 'tolong',
    'pls': 'tolong',
    'pn': 'pun',
    'pnh': 'pernah',
    'pnt': 'penat',
    'pnya': 'punya',
    'pon': 'pun',
    'priority': 'keutamaan',
    'properties': 'harta benda',
    'ptugas': 'petugas',
    'pub': 'kelab malam',
    'pulak': 'pula',
    'puye': 'punya',
    'pwrcuma': 'percuma',
    'pyahnya': 'payahnya',
    'quality': 'kualiti',
    'quit': 'keluar',
    'ramly': 'ramly',
    'rege': 'harga',
    'reger': 'harga',
    'report': 'laporan',
    'resigned': 'meletakkan jawatan',
    'respect': 'hormat',
    'rizal': 'rizal',
    'rosak': 'rosak',
    'rosok': 'rosak',
    'rse': 'rasa',
    'sacked': 'buang',
    'sado': 'tegap',
    'salute': 'sanjung',
    'sam': 'sama',
    'same': 'sama',
    'samp': 'sampah',
    'sbb': 'sebab',
    'sbgai': 'sebagai',
    'sblm': 'sebelum',
    'sblum': 'sebelum',
    'sbnarnya': 'sebenarnya',
    'sbum': 'sebelum',
    'sdg': 'sedang',
    'sebb': 'sebab',
    'sebijik': 'sebiji',
    'see': 'lihat',
    'seen': 'dilihat',
    'selangor': 'selangor',
    'selfie': 'swafoto',
    'sempoi': 'cantik',
    'senaraihitam': 'senarai hitam',
    'seorg': 'seorang',
    'service': 'perkhidmatan',
    'sgt': 'sangat',
    'shared': 'kongsi',
    'shirt': 'kemeja',
    'shut': 'tutup',
    'sib': 'nasib',
    'skali': 'sekali',
    'sket': 'sikit',
    'sma': 'sama',
    'smoga': 'semoga',
    'smpoi': 'cantik',
    'sndiri': 'sendiri',
    'sndr': 'sendiri',
    'sndri': 'sendiri',
    'sne': 'sana',
    'so': 'jadi',
    'sop': 'tatacara pengendalian piawai',
    'sorang': 'seorang',
    'spoting': 'pembintikan',
    'sronok': 'seronok',
    'ssh': 'susah',
    'staff': 'staf',
    'standing': 'berdiri',
    'start': 'mula',
    'steady': 'mantap',
    'stiap': 'setiap',
    'stress': 'stres',
    'student': 'pelajar',
    'study': 'belajar',
    'studycase': 'kajian kes',
    'sure': 'pasti',
    'sykt': 'syarikat',
    'tah': 'entah',
    'taik': 'tahi',
    'takan': 'tak akan',
    'takat': 'setakat',
    'takde': 'tak ada',
    'takkan': 'tak akan',
    'taknak': 'tak nak',
    'tang': 'tentang',
    'tanggungjawab': 'bertanggungjawab',
    'taraa': 'sementara',
    'tau': 'tahu',
    'tbabit': 'terbabit',
    'team': 'pasukan',
    'terbaekk': 'terbaik',
    'teruknye': 'teruknya',
    'tgk': 'tengok',
    'that': 'itu',
    'thinking': 'fikir',
    'those': 'itu',
    'time': 'masa',
    'tk': 'tak',
    'tnggongjwb': 'tanggungjawab',
    'tngok': 'tengok',
    'tngu': 'tunggu',
    'to': 'kepada',
    'tosak': 'rosak',
    'tp': 'tapi',
    'tpi': 'tapi',
    'tpon': 'telefon',
    'transfer': 'pindah',
    'trgelak': 'tergelak',
    'ts': 'tan sri',
    'tstony': 'tan sri tony',
    'tu': 'itu',
    'tuh': 'itu',
    'tula': 'itulah',
    'umeno': 'umno',
    'unfortunately': 'malangnya',
    'unhappy': 'tidak gembira',
    'up': 'naik',
    'upkan': 'naikkan',
    'ur': 'awak',
    'utk': 'untuk',
    'very': 'sangat',
    'viral': 'tular',
    'vote': 'undi',
    'warning': 'amaran',
    'warranty': 'waranti',
    'wassap': 'whatsapp',
    'wat': 'apa',
    'weii': 'wei',
    'well': 'maklumlah',
    'win': 'menang',
    'with': 'dengan',
    'wt': 'buat',
    'x': 'tak',
    'tw': 'tahu',
    'ye': 'ya',
    'yee': 'ya',
    'yg': 'yang',
    'yng': 'yang',
    'you': 'awak',
    'your': 'awak',
    'sakai': 'selekeh',
    'rmb': 'billion ringgit',
    'rmj': 'juta ringgit',
    'rmk': 'ribu ringgit',
    'rm': 'ringgit',
}

In [3]:
permulaan = [
    'bel',
    'se',
    'ter',
    'men',
    'meng',
    'mem',
    'memper',
    'di',
    'pe',
    'me',
    'ke',
    'ber',
    'pen',
    'per',
]

hujung = ['kan', 'kah', 'lah', 'tah', 'nya', 'an', 'wan', 'wati', 'ita']

def naive_stemmer(word):
    assert isinstance(word, str), 'input must be a string'
    hujung_result = [e for e in hujung if word.endswith(e)]
    if len(hujung_result):
        hujung_result = max(hujung_result, key = len)
        if len(hujung_result):
            word = word[: -len(hujung_result)]
    permulaan_result = [e for e in permulaan if word.startswith(e)]
    if len(permulaan_result):
        permulaan_result = max(permulaan_result, key = len)
        if len(permulaan_result):
            word = word[len(permulaan_result) :]
    return word

def build_dataset(words, n_words):
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 3)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary


def classification_textcleaning(string):
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    string = unidecode(string).replace('.', ' . ').replace(',', ' , ')
    string = re.sub('[^A-Za-z ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string.lower()).strip()
    string = [rules_normalizer.get(w, w) for w in string.split()]
    string = [naive_stemmer(word) for word in string]
    return ' '.join([word for word in string if len(word) > 1])


def str_idx(corpus, dic, maxlen, UNK = 3):
    X = np.zeros((len(corpus), maxlen))
    for i in range(len(corpus)):
        for no, k in enumerate(corpus[i].split()[:maxlen][::-1]):
            X[i, -1 - no] = dic.get(k, UNK)
    return X

In [4]:
classification_textcleaning('kerajaan sebenarnya sangat bencikan rakyatnya, minyak naik dan segalanya')

'raja benar sangat benci rakyat minyak naik gala'

In [5]:
df = pd.read_csv('sentiment-data-v2.csv')
Y = LabelEncoder().fit_transform(df.label)
with open('polarity-negative-translated.txt','r') as fopen:
    texts = fopen.read().split('\n')
labels = [0] * len(texts)

with open('polarity-positive-translated.txt','r') as fopen:
    positive_texts = fopen.read().split('\n')
labels += [1] * len(positive_texts)
texts += positive_texts
texts += df.iloc[:,1].tolist()
labels += Y.tolist()

assert len(labels) == len(texts)

In [6]:
import json
with open('bm-amazon.json') as fopen:
    amazon = json.load(fopen)
    
with open('bm-imdb.json') as fopen:
    imdb = json.load(fopen)
    
with open('bm-yelp.json') as fopen:
    yelp = json.load(fopen)
    
texts += amazon['negative']
labels += [0] * len(amazon['negative'])
texts += amazon['positive']
labels += [1] * len(amazon['positive'])

texts += imdb['negative']
labels += [0] * len(imdb['negative'])
texts += imdb['positive']
labels += [1] * len(imdb['positive'])

texts += yelp['negative']
labels += [0] * len(yelp['negative'])
texts += yelp['positive']
labels += [1] * len(yelp['positive'])

In [7]:
import os
for i in [i for i in os.listdir('negative') if 'Store' not in i]:
    with open('negative/'+i) as fopen:
        a = json.load(fopen)
        texts += a
        labels += [0] * len(a)
        
import os
for i in [i for i in os.listdir('positive') if 'Store' not in i]:
    with open('positive/'+i) as fopen:
        a = json.load(fopen)
        texts += a
        labels += [1] * len(a)

In [8]:
x, y = [], []
for i in range(len(texts)):
    s = classification_textcleaning(texts[i])
    if len(s) > 5:
        x.append(s)
        y.append(labels[i])

In [9]:
def convert_sparse_matrix_to_sparse_tensor(X, limit = 5):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    #coo.data[coo.data > limit] = limit
    return tf.SparseTensorValue(indices, coo.col, coo.shape), tf.SparseTensorValue(indices, coo.data, coo.shape)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
bow_chars = CountVectorizer(ngram_range=(3, 5), analyzer='char_wb').fit(x)
delattr(bow_chars, 'stop_words_')

In [12]:
feature_shape = bow_chars.transform(texts[:1]).shape[1]
feature_shape

331938

In [13]:
class Model:
    def __init__(self, vocab_size, learning_rate):
        self.X = tf.sparse_placeholder(tf.int32)
        self.W = tf.sparse_placeholder(tf.int32)
        self.Y = tf.placeholder(tf.int32, [None])
        self.embeddings = tf.Variable(tf.truncated_normal([vocab_size,128]))
        embed = tf.nn.embedding_lookup_sparse(self.embeddings, self.X, self.W, combiner='mean')
        self.logits = tf.layers.dense(embed, 2)
        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1, output_type=tf.int32), self.Y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [14]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(feature_shape, 1e-3)
sess.run(tf.global_variables_initializer())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [15]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'fast-text-char/model.ckpt')

'fast-text-char/model.ckpt'

In [16]:
train_X, test_X, train_Y, test_Y = train_test_split(
    x, y, test_size = 0.2
)

In [17]:
from tqdm import tqdm
import time

batch_size = 32
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 3, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        batch_x = bow_chars.transform(train_X[i : min(i + batch_size, len(train_X))])
        batch_x = convert_sparse_matrix_to_sparse_tensor(batch_x)
        batch_y = train_Y[i : min(i + batch_size, len(train_X))]
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost, _, logits = sess.run(
            [model.accuracy, model.cost, model.optimizer, model.logits],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x[0],
                model.W: batch_x[1],
            },
        )
        assert not np.isnan(cost)
        train_loss += cost
        train_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    pbar = tqdm(range(0, len(test_X), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        batch_x = bow_chars.transform(test_X[i : min(i + batch_size, len(test_X))])
        batch_x = convert_sparse_matrix_to_sparse_tensor(batch_x)
        batch_y = test_Y[i : min(i + batch_size, len(test_X))]
        batch_x_expand = np.expand_dims(batch_x,axis = 1)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x[0],
                model.W: batch_x[1],
            },
        )
        test_loss += cost
        test_acc += acc
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss /= len(train_X) / batch_size
    train_acc /= len(train_X) / batch_size
    test_loss /= len(test_X) / batch_size
    test_acc /= len(test_X) / batch_size

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 19/16751 [00:00<01:29, 186.78it/s, accuracy=0.75, cost=0.653] 

epoch: 0, pass acc: 0.000000, current acc: 0.605330
time taken: 109.6246485710144
epoch: 0, training loss: 0.675434, training acc: 0.580658, valid loss: 0.665955, valid acc: 0.605330



train minibatch loop:   0%|          | 19/16751 [00:00<01:28, 189.25it/s, accuracy=0.594, cost=0.623]

epoch: 1, pass acc: 0.605330, current acc: 0.624076
time taken: 109.83600759506226
epoch: 1, training loss: 0.659840, training acc: 0.614980, valid loss: 0.654714, valid acc: 0.624076



train minibatch loop:   0%|          | 19/16751 [00:00<01:28, 189.51it/s, accuracy=0.594, cost=0.61] 

...

time taken: 107.87554788589478
epoch: 490, training loss: 0.529057, training acc: 0.739200, valid loss: 0.530680, valid acc: 0.739148



train minibatch loop:   0%|          | 20/16751 [00:00<01:27, 191.79it/s, accuracy=0.656, cost=0.605]

time taken: 107.81713533401489
epoch: 491, training loss: 0.529003, training acc: 0.739243, valid loss: 0.530627, valid acc: 0.739156



test minibatch loop: 100%|██████████| 4188/4188 [00:19<00:00, 210.78it/s, accuracy=0.706, cost=0.667]

time taken: 107.8524854183197
epoch: 492, training loss: 0.528949, training acc: 0.739256, valid loss: 0.530575, valid acc: 0.739156

break epoch:493



In [18]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    batch_x = bow_chars.transform(test_X[i : min(i + batch_size, len(test_X))])
    batch_x = convert_sparse_matrix_to_sparse_tensor(batch_x)
    batch_y = test_Y[i : min(i + batch_size, len(test_X))]
    predict_Y += np.argmax(
        sess.run(
            model.logits, feed_dict = {model.X: batch_x[0], model.W: batch_x[1], model.Y: batch_y}
        ),
        1,
    ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 4188/4188 [00:14<00:00, 289.71it/s]


In [19]:
print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['negative','positive']
    )
)

             precision    recall  f1-score   support

   negative       0.75      0.76      0.75     70301
   positive       0.73      0.71      0.72     63700

avg / total       0.74      0.74      0.74    134001



In [24]:
text = classification_textcleaning('kerajaan sebenarnya sangat sayangkan rakyatnya dan benci')
transformed = bow_chars.transform([text])
batch_x = convert_sparse_matrix_to_sparse_tensor(transformed)
sess.run(tf.nn.softmax(model.logits), feed_dict = {model.X: batch_x[0], model.W: batch_x[1]})

array([[0.7648756 , 0.23512435]], dtype=float32)

In [21]:
saver.save(sess, 'fast-text-char/model.ckpt')

'fast-text-char/model.ckpt'

In [22]:
import pickle
with open('vectorizer-sparse-sentiment.pkl','wb') as fopen:
    pickle.dump(bow_chars, fopen)